# **Aula 6 - Séries Temporais**

# TRANSFORMAÇÃO E DIFERENCIAÇÃO

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,6

### **Transformação**

Transformar a série que não tenha distribuição normal em uma série com distribuição aproximadamente normal

**AirPassengers: dados clássicos da companhia aérea Box & Jenkins. Totais mensais de passageiros de companhias aéreas internacionais, 1949 a 1960.**
 

In [3]:
dados = pd.read_csv('AirPassengers.csv', sep=',')

In [4]:
dados

In [5]:
serie = pd.Series(dados['Passengers'].values, index = dados['Month'])
serie

In [6]:
serie.plot()
plt.show()

Tendencia crescente ao longo dos anos e sazonalidade clara, onde de tempos em tempos (regulares) há picos bem marcados. Esses picos começam com uma amplitude pequena e vai aumentando ao passar dos anos.

Verificando a normalidade dos dados 

In [7]:
import scipy.stats as stats

In [8]:
stats.probplot(serie, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Serie aparentemente demonstra problemas de normalidade no começo e um pouco no final, pois é onde ela se distancia da linha.

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

* H0: A série segue uma distribuição aproximadamente normal QUANDO p > 0,05.
* Ha: A série NÃO segue uma distribuição aproximadamente normal QUANDO p < 0,05.

In [9]:
e, p = stats.shapiro(serie)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Como p é menor que 0,05 podemos dizer que a distribuição não é normal.

Para tentarmos resolver esse problema podemos utilizar a transformação logarítmica ou exponencial

Transformação por log (Diminuir variância e melhorar normalidade)

Aplica a função logarítmica em todos os dados da série

In [10]:
serie2 = np.log(serie)
serie2

In [11]:
stats.probplot(serie2, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Observa-se que o início da série a uma aproximação com a linha de referencia, porem o fim da série houve um afastamento maior. Há uma possível na normalidade dos dados.

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

* H0: A série segue uma distribuição aproximadamente normal QUANDO p > 0,05.
* Ha: A série NÃO segue uma distribuição aproximadamente normal QUANDO p < 0,05.

In [12]:
e, p = stats.shapiro(serie2)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Observa-se uma melhoria na normalidade pela proximidade o p-valor com o nivel de significância, porem ainda não temos dados normais

Transformação por raiz cúbica essa transformação é muito utilizada quando a série possui dados com valor zero ou negativos

In [13]:
serie3 = (serie)**(1/3)
serie3

No python sempre que temos o operador de raiz e tivermos números negativos ele fará um tratamento transformando os números em números complexos.

Quando a raiz é quadrada, não existe um numero real que retorne uma raiz negativa (apenas um número complexo), agora se for raiz cubica existiria núneros reais.

Aparece NaN - valor missing

In [14]:
(27)**(1/3)

In [15]:
(-27)**(1/3)

Mostra o número complexo, porem podemos resolver o problema separando o sinal do número(valor absoluto)

In [16]:
np.sign(-27)*abs(-27)**(1/3)

In [17]:
serie_3 = np.sign(serie)*abs(serie)**(1/3)
serie_3

In [18]:
stats.probplot(serie_3, dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Ajusta melhor o final da série , porem no inicio ainda existe um afastamento da linha de referencia. Aparentemente melhorou a normalidade da série

**Teste Shapiro-Wilk**

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

* H0: A série segue uma distribuição aproximadamente normal QUANDO p > 0,05.
* Ha: A série NÃO segue uma distribuição aproximadamente normal QUANDO p < 0,05.

In [19]:
e, p = stats.shapiro(serie_3)
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

Verificando a normalidade dos dados com o histograma

In [20]:
import seaborn as sns
sns.distplot(serie); #serie original

In [21]:
sns.distplot(serie2); #serie transformada com log

In [22]:
sns.distplot(serie3); #serie transformada com a raiz cubica(exponencial)

Buscamos o formato de sino, da curva normal.

### **DIFERENCIAÇÃO**

Transformar uma série não estacionária em uma série estacionária 

In [23]:
import statsmodels.tsa.stattools

Teste KPSS (Kwiatkowski-Phillips-Schmidt-Shin)

Ho = não é estacionário: estatística do teste > valor crítico (p-valor < 0,05)

Ha = é estacionário:  estatística do teste < valor crítico  (p-valor > 0,05)

In [24]:
kpss = statsmodels.tsa.stattools.kpss(serie3)
print('Estatítica do teste: {:.4f}'.format(kpss[0]))
print('p_valor: {:.4f}'.format(kpss[1]))
print('Valores Críticos:')
for chave, valor in kpss[3].items():
   print('{}: {:.4f}'.format(chave, valor))

Como temos a estatística(1.6735) do teste é maior que o valor crítico(0.4630), temos que a série não é estacionária

Teste df (Dickey Fuller)

Ho = não é estacionário: estatística do teste >= valor crítico (p-valor > 0,05)

Ha = é estacionário:  estatística do teste < valor crítico (p-valor < 0,05)

In [25]:
df = statsmodels.tsa.stattools.adfuller(serie3)
print('Estatítica do teste: {:.4f}'.format(df[0]))
print('p_valor: {:.8f}'.format(df[1]))
print('Valores Críticos:')
for chave, valor in df[4].items():
   print('{}: {:.4f}'.format(chave, valor))

Como temos a estatística(-0.8100) do teste é maior que o valor crítico(-2.8840), temos que a série não é estacionária

Fazendo a diferenciação

Lembrando que iremos perder o primeiro valor pois não temos o antecessor para subtrair

In [26]:
serie4 = np.diff(serie3)

In [27]:
plt.plot(serie4)
plt.show()

In [28]:
plt.plot(serie3)
plt.show()

In [29]:
serie.plot()
plt.show()

Desmostrando como é realizada a diferenciação :

Subtração da série por um valor antecessor dela mesma

In [30]:
serie_diff = serie3 - serie3.shift()
#shift -> valor antecessor 

In [31]:
serie_diff.plot()
plt.show()

Teste KPSS (Kwiatkowski-Phillips-Schmidt-Shin)

Ho = não é estacionário: estatística do teste > valor crítico

Ha = é estacionário:  estatística do teste < valor crítico

In [32]:
kpss = statsmodels.tsa.stattools.kpss(serie4)
print('Estatítica do teste: {:.4f}'.format(kpss[0]))
print('p_valor: {:.4f}'.format(kpss[1]))
print('Valores Críticos:')
for chave, valor in kpss[3].items():
   print('{}: {:.4f}'.format(chave, valor))

Como a estatística(0.0266) do teste é menor que o valor crítico(0.4630), temos que a série é estacionária

Caso não tivesse chegado a estacionaridade poderíamos realizar a Diferenciação de segunda ordem.


In [33]:
serie5 = np.diff(serie4)

In [34]:
plt.plot(serie5)
plt.show()

In [35]:
kpss = statsmodels.tsa.stattools.kpss(serie5)
print('Estatítica do teste: {:.4f}'.format(kpss[0]))
print('p_valor: {:.4f}'.format(kpss[1]))
print('Valores Críticos:')
for chave, valor in kpss[3].items():
   print('{}: {:.4f}'.format(chave, valor))

### REVERSÃO DA TRANSFORMAÇÃO E DIFERENCIAÇÃO

Ao realizarmos a transformação e diferenciação das séries temporais alteramos os valores das séries, logo para fazermos previsões temos que realizar a reversão da trasnformação e diferenciação das séries - queremos fazer previsões na escala original

Transformação por log (Este log é o logaritmo natural(ln), tem base e)

In [36]:
serie

In [37]:
serie2 = np.log(serie)
serie2

In [38]:
serie_revertida =  np.e**serie2
serie_revertida

Transformação de logaritmo de base 10

In [39]:
serie2 = np.log10(serie)
serie2

In [40]:
serie_revertida =  10**serie2
serie_revertida

### REVERSÃO DA TRANSFORMAÇÃO POR RAIZ CÚBICA

Transformação por raiz cúbica (quando possui dados com valor zero ou negativos)

In [41]:
serie3 = (serie)**(1/3)
serie3

In [42]:
serie_revertida = serie3**3
serie_revertida

### REVERSÃO DA DIFERENCIAÇÃO

Diferenciação - difereça do valor da serie pelo seu antecessor 

Lembrando que iremos perder o primeiro valor pois não temos o antecessor para subtrair

In [43]:
serie_diferenciada = serie.diff()
serie_diferenciada

In [44]:
serie_revertida = serie.shift(1) + serie_diferenciada
serie_revertida
#shift(1) -> valor antecessor

**Organizando num dataframe**

Fazer a diferenciação dos valores que estão na coluna Passengers e criar uma coluna no Dataframe com o nome: valores_diferenciados

In [45]:
dados['valores_diferenciados'] = dados['Passengers'].diff()
dados

Reverter a diferenciação da coluna valores_diferenciados e criar mais uma coluna com o nome valores_revertidos

In [46]:
dados['valores_revertidos'] = dados['Passengers'].shift(1) + dados['valores_diferenciados']
dados